# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_Gewicht'])
y = df['verschil_Gewicht']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['AfstandTotRestaurant', 'AfstandTotSportterrein', 'AfstandTotKunstijsbaan', 'AfstandTotMuseum', 'AfstandTotPodiumkunstenTotaal', 'AfstandTotBioscoop', 'AfstandTotSauna', 'BeperkingInZien', 'VoldoetAanBeweegrichtlijn', 'UrenMantelzorgPerWeek']
Dropped features: ['AfstandTotAttractie', 'AfstandTotBibliotheek', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotSchool', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotBos', 'AfstandTotCafeED', 'AfstandTotVolkstuin', 'InBezitWoningcorporatie', 'AfstandTotKinderdagverblijf', 'AfstandTotDagrecreatiefTerrein', 'SterfteRelatief', 'NederlandseAntillenEnAruba', 'AfstandTotBrandweerkazerne', 'PercentageOnbewoond', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotZwembad', 'ALandbouwBosbouwEnVisserij', 'AfstandTotBelangrijkOverstapstation', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotTreinstationsTotaal', 'Mantelzorger', 'Rokers', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotApotheek', 'Gemiddel

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -11.67    | 5.415     | 0.04511   | 1.636     |
| 2         | -11.6     | 0.6959    | 0.05126   | 2.078     |
| 3         | -11.64    | 1.222     | 0.01465   | 1.69      |
| 4         | -11.65    | 9.473     | 0.09062   | 3.904     |
| 5         | -11.63    | 2.815     | 0.06129   | 2.657     |
| 6         | -11.64    | 9.167     | 0.07089   | 3.377     |
| 7         | -11.63    | 5.375     | 0.09218   | 2.982     |
| 8         | -11.6     | 0.5399    | 0.008482  | 3.015     |
| 9         | -11.63    | 2.989     | 0.03659   | 3.925     |
| 10        | -11.65    | 2.751     | 0.08992   | 3.122     |
| 11        | -11.61    | 0.3962    | 0.0406    | 2.08      |
| 12        | -11.63    | 2.864     | 0.03342   | 3.865     |
| 13        | -11.61    | 0.8747    | 0.1       | 2.62      |
| 14        | -11.63    | 0.1       | 0.1       | 3.434     |
| 15    

SVR(C=6.774140797265948, gamma=0.004595268341590318, kernel='poly')

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotTreinstationsTotaal', 'AfstandTotZonnebank']
Dropped features: ['AfstandTotAttractie', 'AfstandTotBibliotheek', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotSchool', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotBos', 'AfstandTotCafeED', 'VoldoetAanBeweegrichtlijn', 'AfstandTotVolkstuin', 'InBezitWoningcorporatie', 'AfstandTotKinderdagverblijf', 'UrenMantelzorgPerWeek', 'AfstandTotDagrecreatiefTerrein', 'SterfteRelatief', 'NederlandseAntillenEnAruba', 'AfstandTotBrandweerkazerne', 'AfstandTotPodiumkunstenTotaal', 'PercentageOnbewoond', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotZwembad', 'ALandbouwBosbouwEnVisserij', 'AfstandTotBelangrijkOverstapstation', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotKunstijsbaan', 'AfstandTotBioscoop', 'Mantelzorger', 'Rokers', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotApotheek', 'GemiddeldeWoningwaarde', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotVerblijfsrecreatiefTerr

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -11.44    | 19.46     | 0.9104    | 3.687     | 145.2     |
| 2         | -11.4     | 1.447     | 0.2382    | 5.798     | 229.2     |
| 3         | -11.46    | 4.675     | 0.4554    | 10.92     | 640.3     |
| 4         | -11.45    | 6.888     | 0.4656    | 5.567     | 988.7     |
| 5         | -11.44    | 8.231     | 0.8258    | 6.019     | 251.9     |
| 6         | -11.44    | 16.24     | 0.2215    | 6.145     | 533.6     |
| 7         | -11.45    | 5.879     | 0.471     | 5.657     | 244.2     |
| 8         | -11.45    | 7.195     | 0.2209    | 6.444     | 902.4     |
| 9         | -11.47    | 3.737     | 0.1299    | 6.746     | 601.6     |
| 10        | -11.44    | 15.92     | 0.7102    | 5.549     | 851.0     |
| 11        | -11.49    | 2.852     | 0.5282    | 3.023     | 226.4     |
| 12        | -11.49    | 3.476     | 

RandomForestRegressor(bootstrap=False, max_depth=1,
                      max_features=0.4813287553335581, min_samples_split=8,
                      n_estimators=229)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['AfstandTotCafetariaED']
Dropped features: ['AfstandTotAttractie', 'AfstandTotBibliotheek', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotSchool', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotBos', 'AfstandTotCafeED', 'VoldoetAanBeweegrichtlijn', 'AfstandTotVolkstuin', 'InBezitWoningcorporatie', 'AfstandTotKinderdagverblijf', 'UrenMantelzorgPerWeek', 'AfstandTotDagrecreatiefTerrein', 'SterfteRelatief', 'NederlandseAntillenEnAruba', 'AfstandTotBrandweerkazerne', 'AfstandTotPodiumkunstenTotaal', 'PercentageOnbewoond', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotZwembad', 'ALandbouwBosbouwEnVisserij', 'AfstandTotBelangrijkOverstapstation', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotTreinstationsTotaal', 'AfstandTotKunstijsbaan', 'AfstandTotBioscoop', 'Mantelzorger', 'Rokers', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotApotheek', 'GemiddeldeWoningwaarde', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotVerblijfsrecreatiefTe

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -3.349    | 0.02671   | 79.52     | 0.03777   | 492.8     |
| 2         | -3.351    | 0.0624    | 97.76     | 0.02612   | 694.2     |
| 3         | -3.35     | 0.07469   | 92.64     | 0.008793  | 278.0     |
| 4         | -3.359    | 0.0224    | 81.93     | 0.05514   | 583.7     |
| 5         | -3.342    | 0.03745   | 44.61     | 0.08772   | 390.5     |
| 6         | -3.348    | 0.06595   | 79.09     | 0.04013   | 949.3     |
| 7         | -3.357    | 0.02148   | 45.47     | 0.03208   | 968.0     |
| 8         | -3.362    | 0.08949   | 79.03     | 0.0379    | 951.4     |
| 9         | -3.351    | 0.02997   | 28.95     | 0.03716   | 239.1     |
| 10        | -3.356    | 0.0504    | 21.37     | 0.02854   | 236.3     |
| 11        | -3.35     | 0.09671   | 79.01     | 0.02844   | 739.7     |
| 12        | -3.352    | 0.07882   | 

MLPRegressor(alpha=0.0374455933364084, hidden_layer_sizes=(45,),
             learning_rate_init=0.08772010408759297, max_iter=391)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 2.4324810322722312,
  'mse': 11.407483187566488,
  'rmse': 3.3774965858704413,
  'r2': 0.001819433776709789,
  'adj_r2': -0.07363698051662748},
 {'model': 'Random Forest',
  'mae': 2.444316719603332,
  'mse': 11.431662965555118,
  'rmse': 3.381074232482203,
  'r2': -0.0002963514571576731,
  'adj_r2': -0.07591270631892977},
 {'model': 'Neural Network',
  'mae': 2.440447399049653,
  'mse': 11.489787486367598,
  'rmse': 3.3896589041329217,
  'r2': -0.005382378422272627,
  'adj_r2': -0.08138320616477701},
 {'model': 'Support Vector Machine',
  'mae': 2.3836127999921297,
  'mse': 11.586496334933189,
  'rmse': 3.4038942896237523,
  'r2': -0.013844621288010117,
  'adj_r2': -0.09048514341561353}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 2.7085294488746827,
  'mse': 15.159529909991816,
  'rmse': 3.8935240990639595,
  'r2': -0.006618760105282728,
  'adj_r2': -0.40026855455986254},
 {'model': 'Random Forest',
  'mae': 2.6953939551701644,
  'mse': 15.035564193768085,
  'rmse': 3.8775719456598203,
  'r2': 0.0016127758527318958,
  'adj_r2': -0.3888179821936859},
 {'model': 'Neural Network',
  'mae': 2.7107636581659507,
  'mse': 15.117860906033686,
  'rmse': 3.888169351511542,
  'r2': -0.003851866847497165,
  'adj_r2': -0.39641963600573615},
 {'model': 'Support Vector Machine',
  'mae': 2.696605016873673,
  'mse': 15.800409444438392,
  'rmse': 3.974972886000405,
  'r2': -0.049174259264676445,
  'adj_r2': -0.4594658690329856}]

In [15]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 2.717184553660983
Mean Squared Error: 15.253915202666954
Root Mean Squared Error: 3.905626096116595
R2 Score: -0.012886105257073899
Adjusted R2 Score: -0.4089868168101196


Save best model & features as Pickle

In [20]:
import pickle

# Save model
file_path = 'Saved Models/gewicht.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_gewicht.txt', 'w') as file:
    file.write(f'{best_features_mlr}')